In [1]:
from roboflow import Roboflow
rf = Roboflow(api_key="NjIXpou4o4gsuGClT8hI")
project = rf.workspace("test-l2t0m").project("acne04-bn4vo-xdhi8")
version = project.version(1)
dataset = version.download("yolov11")
                

loading Roboflow workspace...
loading Roboflow project...


In [2]:
import subprocess
import sys

try:
    import ultralytics
except ModuleNotFoundError:
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ultralytics"])
    from ultralytics import YOLO

In [3]:
# 1. 라이브러리 임포트
import os
import shutil
from PIL import Image
import numpy as np
import pandas as pd
from glob import glob
from sklearn.model_selection import train_test_split
from collections import Counter
import torch
import yaml
from ultralytics import YOLO
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import shutil

In [4]:
# 2. GPU 또는 CPU 설정
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

Using device: cuda


In [5]:
# 3. 데이터 경로 설정 (로컬 환경, YOLO11 전용)
# 주의: YOLO는 객체 감지 모델이므로, 데이터셋에 바운딩 박스 레이블이 필요합니다.
# 원본 데이터가 분류용이므로, 여기서는 가상의 레이블 생성을 가정하거나 기존 레이블을 사용합니다.
# 실제로는 각 이미지에 .txt 레이블 파일 (class_id x y w h)이 필요합니다.
BASE_DIR = os.path.join(os.getcwd(), "resized")
IMG_DIR = BASE_DIR
CLASS_DIRS = {
    "normal": os.path.join(BASE_DIR, "normal"),
    "mild": os.path.join(BASE_DIR, "mild"),
    "moderate": os.path.join(BASE_DIR, "moderate"),
    "severe": os.path.join(BASE_DIR, "severe"),
    "very_severe": os.path.join(BASE_DIR, "very_severe")
}
CLASS_ID_MAP = {"normal": 0, "mild": 1, "moderate": 2, "severe": 3, "very_severe": 4}
IMG_EXTS = (".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff", ".webp", ".JPG", ".JPEG", ".PNG")
NUM_CLASSES = 5
EPOCHS = 20
BATCH_SIZE = 16

# YOLO 데이터셋 구조: datasets/train/images, datasets/train/labels 등
DATASET_DIR = os.path.join(BASE_DIR, "yolo_dataset")
os.makedirs(DATASET_DIR, exist_ok=True)
train_dir = os.path.join(DATASET_DIR, 'train')
val_dir = os.path.join(DATASET_DIR, 'val')
test_dir = os.path.join(DATASET_DIR, 'test')
os.makedirs(os.path.join(train_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(train_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(val_dir, 'labels'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(test_dir, 'labels'), exist_ok=True)

# 기존 모델 파일 삭제 (YOLO 모델 저장 경로)
best_model_path = os.path.join(BASE_DIR, 'best_acne_yolo.pt')
if os.path.exists(best_model_path):
    os.remove(best_model_path)
    print(f"Deleted existing model: {best_model_path}")
if os.path.exists(os.path.join(BASE_DIR, 'acne_yolo.pt')):
    os.remove(os.path.join(BASE_DIR, 'acne_yolo.pt'))
    print("Deleted existing model: acne_yolo.pt")

Deleted existing model: c:\Users\Admin\work space\2nd\resized\best_acne_yolo.pt


In [6]:
# 4. 데이터 전처리: CSV 생성 (분류용을 기반으로, YOLO용 레이블 가정)
# 주의: 실제 YOLO 감지에는 바운딩 박스 레이블이 필요. 여기서는 예시로 전체 이미지(1.0 1.0 2.0 2.0)를 클래스별로 가정.
def is_image(p):
    return os.path.isfile(p) and any(p.lower().endswith(ext) for ext in IMG_EXTS)

print(f"BASE_DIR: {BASE_DIR}")
rows = []
for label_name, class_dir in CLASS_DIRS.items():
    if not os.path.isdir(class_dir):
        print(f"[ERROR] {label_name} directory does not exist: {class_dir}")
        continue
    class_files = [p for p in glob(os.path.join(class_dir, "**", "*"), recursive=True) if is_image(p)]
    if len(class_files) == 0:
        print(f"[ERROR] No images found in {label_name} directory: {class_dir}")
        continue
    print(f"[DEBUG] {label_name} files found: {len(class_files)}")
    for fp in class_files:
        rows.append([fp, label_name, CLASS_ID_MAP[label_name]])

if not rows:
    raise ValueError("No images found in any class directories. Check BASE_DIR and image files.")

df = pd.DataFrame(rows, columns=["filepath", "label_name", "label_id"])
out_csv = os.path.join(BASE_DIR, "labels_total.csv")
df.to_csv(out_csv, index=False)
print(f"CSV 생성 완료: {out_csv}")
print("Label distribution:\n", df["label_name"].value_counts())

BASE_DIR: c:\Users\Admin\work space\2nd\resized
[DEBUG] normal files found: 936
[DEBUG] mild files found: 359
[DEBUG] moderate files found: 95
[DEBUG] severe files found: 128
[DEBUG] very_severe files found: 90
CSV 생성 완료: c:\Users\Admin\work space\2nd\resized\labels_total.csv
Label distribution:
 label_name
normal         936
mild           359
severe         128
moderate        95
very_severe     90
Name: count, dtype: int64


In [7]:
# 4. 데이터 전처리: CSV 생성 (분류용을 기반으로, YOLO용 레이블 가정)
# 주의: 실제 YOLO 감지에는 바운딩 박스 레이블이 필요. 여기서는 예시로 전체 이미지(1.0 1.0 2.0 2.0)를 클래스별로 가정.
def is_image(p):
    return os.path.isfile(p) and any(p.lower().endswith(ext) for ext in IMG_EXTS)

print(f"BASE_DIR: {BASE_DIR}")
rows = []
for label_name, class_dir in CLASS_DIRS.items():
    if not os.path.isdir(class_dir):
        print(f"[ERROR] {label_name} directory does not exist: {class_dir}")
        continue
    class_files = [p for p in glob(os.path.join(class_dir, "**", "*"), recursive=True) if is_image(p)]
    if len(class_files) == 0:
        print(f"[ERROR] No images found in {label_name} directory: {class_dir}")
        continue
    print(f"[DEBUG] {label_name} files found: {len(class_files)}")
    for fp in class_files:
        rows.append([fp, label_name, CLASS_ID_MAP[label_name]])

if not rows:
    raise ValueError("No images found in any class directories. Check BASE_DIR and image files.")

df = pd.DataFrame(rows, columns=["filepath", "label_name", "label_id"])
out_csv = os.path.join(BASE_DIR, "labels_total.csv")
df.to_csv(out_csv, index=False)
print(f"CSV 생성 완료: {out_csv}")
print("Label distribution:\n", df["label_name"].value_counts())

BASE_DIR: c:\Users\Admin\work space\2nd\resized
[DEBUG] normal files found: 936
[DEBUG] mild files found: 359
[DEBUG] moderate files found: 95
[DEBUG] severe files found: 128
[DEBUG] very_severe files found: 90
CSV 생성 완료: c:\Users\Admin\work space\2nd\resized\labels_total.csv
Label distribution:
 label_name
normal         936
mild           359
severe         128
moderate        95
very_severe     90
Name: count, dtype: int64


In [8]:
# 5. 데이터 분할 및 밸런싱
train_df, test_df = train_test_split(df, test_size=0.15, stratify=df["label_name"], random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1765, stratify=train_df["label_name"], random_state=42)
print("Train dataset size:", len(train_df))
print("Val dataset size:", len(val_df))
print("Test dataset size:", len(test_df))
print("Test dataset class distribution:\n", test_df["label_name"].value_counts())

targets = {"normal": 1000, "mild": 800, "moderate": 800, "severe": 1000, "very_severe": 1000}
def make_balanced(df, targets):
    rng = np.random.default_rng(42)
    outs = []
    for lab, n in targets.items():
        sub = df[df["label_name"] == lab]
        if len(sub) >= n:
            outs.append(sub.sample(n=n, random_state=42))
        else:
            idx = rng.choice(sub.index.to_numpy(), size=n, replace=True)
            outs.append(df.loc[idx])
    return pd.concat(outs, ignore_index=True).sample(frac=1, random_state=42).reset_index(drop=True)

train_bal = make_balanced(train_df, targets)
print("Train balanced dataset size:", len(train_bal))
train_bal.to_csv(os.path.join(BASE_DIR, "split_train.balanced.csv"), index=False)
val_df.to_csv(os.path.join(BASE_DIR, "split_val.csv"), index=False)
test_df.to_csv(os.path.join(BASE_DIR, "split_test.csv"), index=False)
print("Train:\n", train_bal["label_name"].value_counts())
print("Val:\n", val_df["label_name"].value_counts())
print("Test:\n", test_df["label_name"].value_counts())

Train dataset size: 1124
Val dataset size: 242
Test dataset size: 242
Test dataset class distribution:
 label_name
normal         141
mild            54
severe          19
very_severe     14
moderate        14
Name: count, dtype: int64
Train balanced dataset size: 4600
Train:
 label_name
very_severe    1000
normal         1000
severe         1000
moderate        800
mild            800
Name: count, dtype: int64
Val:
 label_name
normal         141
mild            54
severe          19
very_severe     14
moderate        14
Name: count, dtype: int64
Test:
 label_name
normal         141
mild            54
severe          19
very_severe     14
moderate        14
Name: count, dtype: int64


In [9]:
# 6. YOLO 데이터셋 폴더 구조 생성 및 레이블 파일 생성
# 주의: 실제 바운딩 박스 대신 더미 데이터 (전체 이미지: class_id 0.5 0.5 1.0 1.0) 사용. 실제 데이터로 교체 필요.
def organize_yolo_folders(csv_df, dest_folder):
    for _, row in csv_df.iterrows():
        label_id = row['label_id']
        src_path = row['filepath']
        img_dest_dir = os.path.join(dest_folder, 'images')
        label_dest_dir = os.path.join(dest_folder, 'labels')
        dest_img_path = os.path.join(img_dest_dir, os.path.basename(src_path))
        dest_label_path = os.path.join(label_dest_dir, os.path.splitext(os.path.basename(src_path))[0] + '.txt')
        if os.path.exists(src_path):
            shutil.copy(src_path, dest_img_path)
            # 더미 레이블 생성 (전체 이미지 커버)
            with open(dest_label_path, 'w') as f:
                f.write(f"{label_id} 0.5 0.5 1.0 1.0\n")

organize_yolo_folders(train_bal, train_dir)
organize_yolo_folders(val_df, val_dir)
organize_yolo_folders(test_df, test_dir)
print("YOLO 폴더 구조 및 레이블 생성 완료!")

YOLO 폴더 구조 및 레이블 생성 완료!


In [10]:
# 7. YOLO 데이터 YAML 파일 생성
data_yaml = {
    'path': DATASET_DIR,
    'train': 'train/images',
    'val': 'val/images',
    'test': 'test/images',
    'nc': NUM_CLASSES,
    'names': list(CLASS_ID_MAP.keys())
}
yaml_path = os.path.join(BASE_DIR, 'data.yaml')
with open(yaml_path, 'w') as f:
    yaml.dump(data_yaml, f)
print(f"YAML 파일 생성 완료: {yaml_path}")
print("클래스 매핑:", data_yaml['names'])

YAML 파일 생성 완료: c:\Users\Admin\work space\2nd\resized\data.yaml
클래스 매핑: ['normal', 'mild', 'moderate', 'severe', 'very_severe']


In [11]:
# 8. 모델 정의: YOLO11 전이 학습
model = YOLO('yolo11n.pt')  # YOLO11 나노 버전 사전 훈련 모델 로드 (감지용)
model.to(DEVICE)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [12]:
# 9. 학습 설정
# YOLO는 내장된 학습 함수 사용. 하이퍼파라미터는 train 메서드에서 설정.
# 조기 종료는 Ultralytics에서 지원되지 않으므로, 수동으로 구현하거나 생략.
best_model_path = os.path.join(BASE_DIR, 'best_acne_yolo.pt')

In [13]:
# 10. 학습 루프 (YOLO 스타일)
results = model.train(
    data=yaml_path,
    epochs=EPOCHS,
    imgsz=224,  # 원본과 비슷하게 224x224
    batch=BATCH_SIZE,
    device=DEVICE,
    name='acne_yolo',
    patience=5,  # 조기 종료 patience
    project=BASE_DIR,
    exist_ok=True
)
# 학습 후 최적 모델 저장
model.save(best_model_path)
print(f"Best model saved at: {best_model_path}")

New https://pypi.org/project/ultralytics/8.3.201 available  Update with 'pip install -U ultralytics'
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=c:\Users\Admin\work space\2nd\resized\data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=20, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=224, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=acne_yolo, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=

In [14]:
# 11. 평가: 테스트 데이터셋
val_results = model.val(data=yaml_path, split='test')
print("Test Metrics:")
print(val_results)

# 혼동 행렬 및 분류 보고서 (YOLO 결과에서 추출, 필요 시 커스텀)
# YOLO는 mAP, precision 등을 제공. 분류 스타일로 변환 필요.
# 여기서는 간단히 mAP 출력
print(f"mAP@0.5: {val_results.box.map50}")

Ultralytics 8.3.200  Python-3.13.7 torch-2.7.1+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,583,127 parameters, 0 gradients, 6.3 GFLOPs
val: Fast image access  (ping: 0.30.1 ms, read: 1.70.2 MB/s, size: 11.5 KB)
val: Scanning C:\Users\Admin\work space\2nd\resized\yolo_dataset\test\labels.cache... 242 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 242/242 275.6Kit/s 0.0s
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 16/16 12.4it/s 1.3s0.1s
                   all        242        242      0.642      0.807      0.757      0.682
                normal        141        141      0.985          1      0.995      0.959
                  mild         54         54       0.76          1      0.943      0.838
              moderate         14         14      0.331      0.389      0.327      0.281
                severe         19         19      0.506      0.862      0.688    

In [15]:
# 12. 손실 및 메트릭 시각화 (YOLO 결과에서 추출)
# YOLO는 results.csv 등을 생성. 이를 로드하여 플롯.
results_csv = os.path.join(BASE_DIR, 'acne_yolo', 'results.csv')  # 실제 경로 확인 필요
if os.path.exists(results_csv):
    results_df = pd.read_csv(results_csv)
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    plt.plot(results_df['train/box_loss'], label='Train Box Loss')
    plt.plot(results_df['val/box_loss'], label='Val Box Loss')
    plt.title('Box Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(results_df['metrics/precision(B)'], label='Precision')
    plt.plot(results_df['metrics/recall(B)'], label='Recall')
    plt.title('Precision & Recall')
    plt.legend()
    plt.show()
else:
    print("Results CSV not found. Visualization skipped.")

<Figure size 1000x500 with 2 Axes>